# Sequence to Sequence (a.k.a. seq2seq)

**학습목표**
* Encoder Decoder 구조를 이해하고 구현할 줄 안다.
* Seq2Seq에 필요한 전처리를 이해한다.
* **데이터 부족**과, **긴 문장**을 겪어본다.

![이런거](https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png)
---------------------------------
edu.rayleigh@gmail.com
Special Thanks to : 숙번님 ( [봉수골 개발자 이선비](https://www.youtube.com/channel/UCOAyyrvi7tnCAz7RhH98QCQ) )

In [ ]:
url = 'https://raw.githubusercontent.com/kitae104/New_Python/master/Chatbot/data/ChatbotData%20.csv'

In [ ]:
import pandas as pd
temp = pd.read_csv(url, usecols=[0,1])
temp.shape

(11823, 2)

In [ ]:
temp.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [ ]:
Q_sent = temp['Q'].tolist()
A_sent = temp['A'].tolist()

In [ ]:
print(Q_sent[1000])
print(A_sent[1000])

노래방 걸 거 같은데 뭐 부르지
달달한 노래요.


# 데이터 준비
0. 단어와 구두점 사이 공백 만들기
1. sos 와 eos
1. tokenizing, idx_seq, padding

## 0. 단어와 구두점 사이 공백 만들기


In [ ]:
import unicodedata
import re
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def preprocessor(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [ ]:
preprocessor("나는 멋있어.")

'나는 멋있어 . '

In [ ]:
A_sent = [ preprocessor(sent) for sent in A_sent ]
Q_sent = [ preprocessor(sent) for sent in Q_sent ]

In [ ]:
print(A_sent[1000])
print(Q_sent[1000])

달달한 노래요 . 
노래방 걸 거 같은데 뭐 부르지


## 1. sos 와 eos
1. sos : start of speech
2. eos : end of speech

In [ ]:
######################
### Your Code here ###
######################

## 답변 문장 전 후에 <sos>와 <eos>를 추가할 것
## 띄어쓰기 주의!

A_sent = [f"<sos> {A} <eos>" for A in A_sent]
A_sent[1000]

'<sos> 달달한 노래요 .  <eos>'

## 2. Tokenizing, idx_seq, padding

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
######################
### Your Code here ###
######################

# Tokenizing    # 한국어는 lower = False
tokenizer_A = Tokenizer(filters="", lower=False)
tokenizer_A.fit_on_texts(A_sent)
tokenizer_Q = Tokenizer(filters="", lower=False)
tokenizer_Q.fit_on_texts(Q_sent)

In [ ]:
######################
### Your Code here ###
######################

# Index Sequence
A_seq = tokenizer_A.texts_to_sequences(A_sent)
Q_seq = tokenizer_Q.texts_to_sequences(Q_sent)

print(A_seq[1000])
print(Q_seq[1000])

[1, 1783, 3475, 3, 2]
[650, 142, 5, 77, 44, 2835]


In [ ]:
######################
### Your Code here ###
######################
## 최대 문장 길이에 맞춰지도록 할 것.
# padding
A_pad = pad_sequences(A_seq, padding='post') # 최대 문장 길이에 패딩에 맞춰지게 됨.
Q_pad = pad_sequences(Q_seq)

print(Q_pad.shape)
print(A_pad.shape)

(11823, 16)
(11823, 26)


In [ ]:
# tokenizer에서 0 index가 구성되어있지 않지만, 
# pad_sequence에서 pad의 의미로 0을 사용하고 있어서, 전체 사이즈를 구할 때, +1을 해준다.

A_vocab_size = len(tokenizer_A.word_index) + 1
Q_vocab_size = len(tokenizer_Q.word_index) + 1
print("질문 단어 집합의 크기: {:d}\n답변 단어 집합의 크기: {:d}".format(Q_vocab_size, A_vocab_size))

질문 단어 집합의 크기: 13418
답변 단어 집합의 크기: 9855


# 모델링!

1. 모든 임베딩 레이어는 128개 차원으로 구성.
2. 인코더도 디코더도 GRU, 히든스테이트 512로 구성.
3. 디코더의 GRU 뒤에는 Fully Conneceted layer 사용. 노드 512개
4. 적절한 아웃풋레이어
    * 매 시점, 가장 적절한 단어가 무엇일지 분류 한다고 생각하면 됨!

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU

In [ ]:
######################
### Your Code here ###
######################

# 혹시 이미 그려둔 그래프가 있다면 날려줘!
tf.keras.backend.clear_session()

# 질문 단어 집합의 크기 : 13418, (11823, 16)
# 답변 단어 집합의 크기 : 9855, (11823, 26)
# 디코더의 인풋으로 넣을때는 맨 뒤의 <eos>를 떼고 길이 103의 문장을
# 디코더의 아웃풋은 맨 앞의 <eos>를 떼고 길이 103의 문장으로 준비해야 함.

# Encoder
enc_X = tf.keras.layers.Input(shape=[Q_pad.shape[1]])
enc_E = tf.keras.layers.Embedding(Q_vocab_size, 128)(enc_X) # 토큰수, 차원수
enc_S_full, enc_S = tf.keras.layers.GRU(512, return_sequences=True, return_state=True)(enc_E)
## 이제는 enc_S_full을 쓴다!

# Decoder
dec_X = tf.keras.layers.Input(shape=[A_pad.shape[1]-1])
dec_E = tf.keras.layers.Embedding(A_vocab_size, 128)(dec_X) # 토큰수, 차원수
dec_H = tf.keras.layers.GRU(512, return_sequences=True)(dec_E, initial_state=enc_S)

#####################
## Attention layer ##
#####################
key = enc_S_full  # 인코더의 히든스테이트를 key로 활용한다. 
value = enc_S_full  # 인코더의 히든스테이트를 value로 활용한다. 
query = dec_H  # 디코더의 히든스테이트를 query로 활용한다.

# 1. 어텐션 스코어(Attention Score)를 구한다.
score = tf.matmul(query, key, transpose_b=True)

# 2. 소프트맥스(softmax) 함수를 통해 어텐션 분포(Attention Distribution)를 구한다.
att_dist = tf.nn.softmax(score, axis=-1)

# 3. 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값(Attention Value)을 구한다.
att_value = tf.matmul(att_dist, value)

### 사실은, 아래 한줄의 코드로도 가능.
# att_value = tf.keras.layers.Attention()([query, key])

# 4. 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결한다.(Concatenate)

dec_H = tf.keras.layers.Concatenate()([att_value, dec_H])
# 5. 출력층 연산의 입력이 되는 dec_H를 계산.
dec_H = tf.keras.layers.Dense(512, activation='tanh')(dec_H)

dec_Y = tf.keras.layers.Dense(Q_vocab_size, activation="softmax")(dec_H)
model = tf.keras.models.Model([enc_X, dec_X], dec_Y)
# 텍스트는 index이고(원핫인코딩을 안했고)
# 아웃풋레이어는 분류문제 처럼 노드가 준비되어 있다면
# sparse categorical crossentropy
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 16)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 16, 128)      1717504     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 25, 128)      1261440     input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
######################
### Your Code here ###
######################
## 학습 시킬 것!

# decoder의 인풋은 마지막 <eos>를 뗀다.
# decoder의 아웃풋 학습시엔 처음의 <sos>를 뗀다.
model.fit([Q_pad, A_pad[:, :-1]], A_pad[:, 1:], shuffle=True, 
          batch_size=128, epochs=40)

Epoch 1/40
93/93 [==============================] - 17s 84ms/step - loss: 2.9734 - accuracy: 0.6901
Epoch 2/40
93/93 [==============================] - 8s 84ms/step - loss: 1.4790 - accuracy: 0.7810
Epoch 3/40
93/93 [==============================] - 8s 85ms/step - loss: 1.3161 - accuracy: 0.8078
Epoch 4/40
93/93 [==============================] - 8s 85ms/step - loss: 1.2303 - accuracy: 0.8149
Epoch 5/40
93/93 [==============================] - 8s 86ms/step - loss: 1.1881 - accuracy: 0.8188
Epoch 6/40
93/93 [==============================] - 8s 86ms/step - loss: 1.1364 - accuracy: 0.8206
Epoch 7/40
93/93 [==============================] - 8s 87ms/step - loss: 1.1039 - accuracy: 0.8237
Epoch 8/40
93/93 [==============================] - 8s 88ms/step - loss: 1.0562 - accuracy: 0.8281
Epoch 9/40
93/93 [==============================] - 8s 89ms/step - loss: 1.0217 - accuracy: 0.8307
Epoch 10/40
93/93 [==============================] - 8s 90ms/step - loss: 0.9693 - accuracy: 0.8358
Epoch 11

In [ ]:
import numpy as np

# 질문 단어 집합의 크기 : 13418, (11823, 16)
# 답변 단어 집합의 크기 : 9855, (11823, 26)

def translate(Q):
    # A => index => pad
    Q_seq = tokenizer_Q.texts_to_sequences([Q])
    Q_pad = tf.keras.preprocessing.sequence.pad_sequences(Q_seq, maxlen=16)

    A = []
    for n in range(26-1):
        # Q => index => pad
        A_seq = tokenizer_A.texts_to_sequences([['<sos>'] + A])
        A_pad = tf.keras.preprocessing.sequence.pad_sequences(A_seq, maxlen=26-1)
        A_next = model.predict([Q_pad, A_pad])

        # onehot -> index -> word
        A = [tokenizer_A.index_word[i] for i in np.argmax(A_next[0], axis=1) if i != 0]
        # 번역된 word 선택
        A = A[:n+1]
        
        if A[-1] == '<eos>':
            break

    return A

In [ ]:
import random

# 랜덤 10개
indices = list(range(3648))
random.shuffle(indices)

for n in indices[:10]:
    print(f"한국어: {Q_sent[n]}\n영어: {A_sent[n]}")
    print(f"번역: {' '.join(translate(Q_sent[n])[:-1])}")
    print()

한국어: 사업 구상하고 있어
영어: <sos> 성공하길 바랍니다 .  <eos>
번역: 지금은 괜찮길 사람이에요 . <eos>

한국어: 결혼준비하는데 돈 얼마나 드나
영어: <sos> 욕심에 따라 천지 차이일 거예요 .  <eos>
번역: 일이 수도 할 . <eos> <eos>

한국어: 엿같다 . 
영어: <sos> 벗어나는 게 좋겠네요 .  <eos>
번역: 벗어나는 좋은 것 거예요 .

한국어: 엄마한테 막말했어
영어: <sos> 더 후회하기 전에 사과하세요 .  <eos>
번역: 좋은 곳으로 갈 바뀔 것도 수도 <eos> 생각해요

한국어: 다음에 봐
영어: <sos> 잘가요 .  <eos>
번역: 안녕히 . ! <eos>

한국어: 목욕탕 가야지
영어: <sos> 시원하게 씻고 오세요 .  <eos>
번역: 시원하게 좋은 . 것 .

한국어: 반 배정 좀 잘 됐으면 좋겠다
영어: <sos> 잘 되길 바랍니다 .  <eos>
번역: 많이 잘 해놨나봐요 살 .

한국어: 오늘 보름달이다 . 
영어: <sos> 소원을 비세요 .  <eos>
번역: 소원을 필요해요 <eos> .

한국어: 연금 믿어도 될까
영어: <sos> 없는 것보다 나을 거예요 .  <eos>
번역: 원하는 게 만큼 중요한 좋아요 . <eos>

한국어: 내가 제정신이 아니다
영어: <sos> 